In [33]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [34]:
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name="diabetes_classification")

In [35]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='diabetes_classification')
df = dataset.to_pandas_dataframe()

In [36]:
df.isna().sum()

p_id                     0
no_times_pregnant        0
glucose_concentration    0
blood_pressure           0
skin_fold_thickness      0
serum_insulin            0
bmi                      0
diabetes pedigree        0
age                      0
diabetes                 0
dtype: int64

In [37]:
X = df.iloc[: , :-1]
y = df.iloc[: , -1]

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

lr=LogisticRegression()
lr.fit(X_train, y_train)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
y_pred=lr.predict(X_train)

In [41]:
from sklearn import metrics
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

Accuracy Score:- 0.7087576374745418
F1 Score:- 0.4837545126353791
Average Precision Score:- 0.44803180160693246
Log Loss:- 10.059224861836508
Precision Score:- 0.6090909090909091
Recall Score:- 0.40119760479041916
ROC-AUC Score:- 0.6342407777038515


In [42]:
import joblib
import os
model_file_name = 'diabetes_logistic_regression.pkl'

# save model in the outputs folder so it automatically get uploaded
with open(model_file_name, "wb") as file:
    joblib.dump(value=lr, filename=os.path.join('./outputs/',
                                                    model_file_name))

In [43]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name="diabetes_logistic_regression", model_path="./outputs/diabetes_logistic_regression.pkl")

Registering model diabetes_logistic_regression


In [44]:
%%writefile score.py
import json
import os
import joblib
import numpy as np

# from inference_schema.schema_decorators import input_schema, output_schema
# from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    print("This is init")
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'diabetes_logistic_regression.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)



# @input_schema('data', NumpyParameterType(np.array([[0.1, 1.2, 2.3, 3.4, 4.5, 5.6, 6.7, 7.8, 8.9, 9.0]])))
# @output_schema(NumpyParameterType(np.array([4429.929236457418])))
def run(data):
    data = np.array(json.loads(data)['data'])
    # Use the model object loaded by init().
    result = model.predict(data)

    # You can return any JSON-serializable object.
    return result.tolist()

Overwriting score.py


In [31]:
%%writefile score.py
import json
import os
import joblib

def init():
    global model
    print("This is init")
    model_filename = 'diabetes_logistic_regression.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)


def run(data):
    test = json.loads(data)
    result = model.predict(test)
    print(f"received data {test}")
    # return "raw: {}, json.loads: {}, result: {}".format(data, test, result)
    return result.tolist()

Overwriting score.py


In [45]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
inference_config = InferenceConfig(
    environment=env,
    entry_script="score.py",
)

In [46]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [47]:
service = Model.deploy(
    ws,
    "diabetes-logistic-regression",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-09 10:31:23+00:00 Creating Container Registry if not exists.
2021-12-09 10:31:23+00:00 Registering the environment.
2021-12-09 10:31:24+00:00 Use the existing image.
2021-12-09 10:31:24+00:00 Generating deployment configuration.
2021-12-09 10:31:25+00:00 Submitting deployment to compute.
2021-12-09 10:31:28+00:00 Checking the status of deployment diabetes-logistic-regression.

-----------------------------------------------------------------------------------
**### Eperimentation**

In [24]:
import json
input_payload = json.dumps({
    'data': X_test[0:2].values.tolist()
})

output = service.run(json.dumps({"data" : [[288, 1, 119, 86, 39, 220, 45.6, 0.808, 29]]}))

print(output)

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '249', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 08 Dec 2021 19:42:11 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': '4e6046f8-6e7e-4c86-a88f-c2d01db025a5', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Expected 2D array, got scalar array instead:\narray={"data": [[288, 1, 119, 86, 39, 220, 45.6, 0.808, 29]]}.\nReshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '249', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 08 Dec 2021 19:42:11 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': '4e6046f8-6e7e-4c86-a88f-c2d01db025a5', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Expected 2D array, got scalar array instead:\narray={"data": [[288, 1, 119, 86, 39, 220, 45.6, 0.808, 29]]}.\nReshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '2